In [3]:
%load_ext autoreload
%autoreload 2 

In [4]:
import os
import sys

In [5]:
%env CUDA_VISIBLE_DEVICES=3

env: CUDA_VISIBLE_DEVICES=3


In [6]:
sys.path.append(os.path.normpath(os.path.join(os.getcwd(), '..')))
sys.path.append(os.path.normpath(os.path.join(os.getcwd(), '..', 'external', 'MiDaS')))

In [7]:
from src.depth.midas_hkrm import MidasHKRMNet
from src.objects.features import GeneralizedRCNNObjectDetector

In [8]:
from detectron2.evaluation import COCOEvaluator
from detectron2.data import MetadataCatalog
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog, build_detection_test_loader, DatasetMapper
from detectron2.evaluation import inference_on_dataset
import torch

from src.utils.objects_utils import get_hkrm_model, construct_config
import src.objects
from detectron2.modeling import build_model
from src.datasets import HRWSI, ApolloScape, RedWeb
import matplotlib.pyplot as plt
import numpy as np

In [9]:
cfg = construct_config()
cfg.MODEL.WEIGHTS = '/runai-ivrl-scratch/students/2021-fall-sp-jellouli/output/model_final.pth'

In [10]:
detector = GeneralizedRCNNObjectDetector(cfg)

In [11]:
model = MidasHKRMNet(detector, 10, device='cuda')

Using cache found in /runai-ivrl-scratch/students/2021-fall-sp-jellouli/.cache/torch/hub/facebookresearch_WSL-Images_main


In [12]:
hrwsi = iter(HRWSI())

In [13]:
img, label = next(hrwsi)

In [14]:
apollo = iter(ApolloScape())
redweb = iter(RedWeb())

In [15]:
trans = torch.hub.load("intel-isl/MiDaS", "transforms")

Using cache found in /runai-ivrl-scratch/students/2021-fall-sp-jellouli/.cache/torch/hub/intel-isl_MiDaS_master


In [16]:
img, _ = next(apollo)

In [47]:
trans.default_transform(img).shape

torch.Size([1, 3, 128, 384])

In [27]:
model([img], trans.default_transform(img).to('cuda'))

tensor([[[0.2460, 0.2361, 0.2446,  ..., 0.2401, 0.2370, 0.2434],
         [0.1868, 0.1788, 0.1683,  ..., 0.2102, 0.2139, 0.2507],
         [0.1828, 0.1594, 0.1532,  ..., 0.2153, 0.2100, 0.2505],
         ...,
         [0.2422, 0.2346, 0.1984,  ..., 0.2401, 0.2431, 0.3019],
         [0.2611, 0.2571, 0.2188,  ..., 0.2530, 0.2539, 0.3079],
         [0.2320, 0.2750, 0.2503,  ..., 0.2556, 0.2629, 0.2907]]],
       device='cuda:0', grad_fn=<SqueezeBackward1>)

In [25]:
img.shape

(668, 900, 3)

In [21]:
import torch.nn.functional as F

In [ ]:
torch.randn((1, 10))

In [26]:
F.interpolate

<function torch.nn.functional.interpolate(input, size=None, scale_factor=None, mode='nearest', align_corners=None, recompute_scale_factor=None)>

In [49]:
a = torch.randn((1,1024))

In [53]:
import torchvision.transforms.functional as TF

In [58]:
TF.resize(a.reshape(1, 32, 32), size= (128, 256)).shape

torch.Size([1, 128, 256])

In [21]:
a = torch.zeros((1, 1024))

In [22]:

b = torch.ones((1,1024))

In [23]:

c = torch.cat((a,b)).reshape(2, 32 ,32)

In [66]:
c.shape

torch.Size([2, 32, 32])

In [68]:
TF.resize(c, (64, 64)).shape

torch.Size([2, 64, 64])

In [25]:
model._resize_features(c, (32,28)).shape

torch.Size([2, 32, 28])